<a href="https://colab.research.google.com/github/nikopj/SummerML/blob/master/Day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 2: Linear Regression

## Outline:
- Motivating Example
- Linear Model
- Least Squares Fit Problem
- Sample Mean and Variance
- LS Fit Solution